# 02. Entraînement et Sélection du Modèle Final

Ce notebook implémente le workflow complet validé :
1. **Benchmark Initial** : Comparer 4 modèles de base (Dummy, LogReg, RF, XGB) sur les datasets V1 et V2 pour choisir le meilleur dataset.
2. **Sélection Dataset & Split** : On fixe le split (Train/Val/Test) du meilleur dataset identifié.
3. **Optimisation LightGBM** : On optimise LightGBM uniquement sur ce dataset.
4. **Cross-Validation** : On valide la robustesse avec une CV 5-folds sur le Train set.
5. **Entraînement Final** : On réentraîne le modèle final sur Train+Val (optionnel) ou juste Train, puis on évalue sur Test.
6. **Sauvegarde** : Modèle pour serving MLflow.

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import sys
import os
import joblib
import shutil

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path: sys.path.append(project_root)

from src.model_utils import (
    get_train_val_test_split,
    train_dummy, train_logistic_regression, train_random_forest, 
    train_xgboost, train_lightgbm,
    train_lightgbm_cv, optimize_lightgbm, COST_WEIGHTS
)

mlflow.set_tracking_uri("../mlruns")
mlflow.set_experiment("Credit_Scoring_Final_Workflow")

In [ ]:
# Chargement V1/V2
X_v1 = pd.read_pickle('../data/processed/X_prepared_v1.pkl')
y_v1 = pd.read_pickle('../data/processed/y_prepared_v1.pkl')
X_v2 = pd.read_pickle('../data/processed/X_prepared_v2.pkl')
y_v2 = pd.read_pickle('../data/processed/y_prepared_v2.pkl')

def clean_cols(df):
    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
    return df
X_v1 = clean_cols(X_v1)
X_v2 = clean_cols(X_v2)

## 1. Benchmark Rapide (Base Models)

In [ ]:
results = []

# On stocke les données splittées pour pouvoir réutiliser celles du gagnant
splits = {}

for name, X, y in [("v1", X_v1, y_v1), ("v2", X_v2, y_v2)]:
    print(f"--- Benchmarking Dataset {name} ---")
    Xt, yt, Xv, yv, Xte, yte = get_train_val_test_split(X, y)
    splits[name] = (Xt, yt, Xv, yv, Xte, yte)
    
    # Dummy
    _, m = train_dummy(Xt, yt, Xv, yv, Xte, yte, name)
    results.append({"Data": name, "Model": "Dummy", **m})
    
    # LogReg
    _, m = train_logistic_regression(Xt, yt, Xv, yv, Xte, yte, name)
    results.append({"Data": name, "Model": "LogReg", **m})
    
    # RF
    _, m = train_random_forest(Xt, yt, Xv, yv, Xte, yte, name)
    results.append({"Data": name, "Model": "RF", **m})
    
    # XGB
    _, m = train_xgboost(Xt, yt, Xv, yv, Xte, yte, name)
    results.append({"Data": name, "Model": "XGB", **m})
    
    # LightGBM (Baseline)
    _, m = train_lightgbm(Xt, yt, Xv, yv, Xte, yte, name)
    results.append({"Data": name, "Model": "LightGBM", **m})

df_res = pd.DataFrame(results).sort_values("business_cost")
display(df_res[["Data", "Model", "business_cost", "auc", "val_best_cost"]])

## 2. Sélection du Dataset Gagnant pour l'Optimisation
On prend le dataset qui a donné le meilleur score LightGBM (le modèle cible).

In [ ]:
lgbm_res = df_res[df_res["Model"] == "LightGBM"].sort_values("business_cost")
best_data_name = lgbm_res.iloc[0]["Data"]
print(f"Dataset sélectionné pour optimisation LightGBM : {best_data_name}")

# Récupération des splits EXISTANTS (pas de re-split)
X_train, y_train, X_val, y_val, X_test, y_test = splits[best_data_name]
print(f"Shape Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

## 3. Optimisation Optuna LightGBM

In [ ]:
best_params = optimize_lightgbm(X_train, y_train, X_val, y_val, n_trials=30)

final_params = best_params.copy()
final_params.update({
    "metric": "custom", "objective": "binary", "verbosity": -1,
    "boosting_type": "gbdt", "random_state": 42, "n_jobs": -1,
    "class_weight": COST_WEIGHTS, "n_estimators": 1000
})

## 4. Cross-Validation de Confirmation

In [ ]:
# On vérifie la stabilité sur le Train Set complet (si on voulait concatener train+val on pourrait, mais gardons Train)
mean_auc, mean_cost = train_lightgbm_cv(X_train, y_train, params=final_params)

## 5. Entraînement Final & Evaluation Test

In [ ]:
# On entraîne le modèle final sur Train (avec Val pour early stopping) 
# et on évalue sur Test. C'est CE modèle qui part en prod.
model_final, metrics_final = train_lightgbm(
    X_train, y_train, X_val, y_val, X_test, y_test, 
    dataset_name=f"{best_data_name}_OPTI_FINAL", 
    params=final_params
)

print("METRICS FINALES SUR TEST:", metrics_final)

In [ ]:
if not os.path.exists("../models"):
    os.makedirs("../models")

joblib.dump(model_final, "../models/best_model.pkl")

path_serving = "../models/final_model"
if os.path.exists(path_serving): shutil.rmtree(path_serving)
mlflow.lightgbm.save_model(model_final, path_serving)
print("Modèle prêt pour Docker!")